Point-wise outliers usually occur when there are potential system failures or small glitches in the time series. This kind of outlier exists on single data point in global (comparing to the data points in the whole time series) or local manner (comparing to neighboring points). Global outliers are usually obvious, the common practice to detect global outlier is to obtain the statistical values (e.g., min/max/mean/standard deviation)of the dataset and set a threshold value for detecting the anomalous points. Local outliers usually occur in certain context, data points with same value will not be identified as an outlier if it is not exhibited in the specific context. The common strategy to detect local outliers is to identify the context (via seasonality trend decomposition, auto-correlation), then apply statistical/machine learning methods (e.g., AutoRegression, IsolationForest, OneClassSVM) to detect the outliers.


Pattern-wise outliers usually appear when there are abnormal behaviors existing in the data. Pattern outliers refer to the subsequences (consecutive points) of the time series data whose behavior is unusual comparing to other subsequences. Common practices to detect pattern outliers including discords analysis (e.g., matrix profile [6], HotSAX [7]), and subsequence clustering [4]. Discords analysis leverages a sliding window to segment time series into multiple subsequences and computes the distances (e.g., Euclidean distance) between the subsequences to find the discords in the time series data. Subsequence clustering also applies subsequence segmentation to the time series data and adopts subsequences as features for each time point, where the size of sliding window is the number of features. Then, unsupervised machine learning methods such as clustering (e.g., KMeans, PCA) or point-wise outlier detection algorithms are adopted to detect the pattern outliers.


System-wise outliers constantly happen when one of many systems are in abnormal state where a system is defined as a multivariate time series data. The goal of detecting system outliers is to find out a system that is under abnormal state from many of the similar systems. For example, detecting an abnormal manufacturing line from a factory with multiple manufacturing lines. The common approach to detect this kind of outliers is to perform both point-wise and pattern-wise outlier detection to get the outlierness score for each time point/subsequence, then adopt ensemble techniques to generate an overall outlieness score for each system for comparison and detection.




### Because we have to label each id as either an anomaly or not, we will use point-wise outliers

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import figure
from matplotlib.pyplot import suptitle
import matplotlib.style as style
from IPython.display import display, HTML

from pyod.models.combination import aom, moa, average, maximization, median
from pyod.utils.utility import standardizer
from pyod.utils.data import evaluate_print
import warnings
%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
warnings.filterwarnings('ignore') 

# Set some parameters to get good visuals - style to ggplot and size to 15,10

pd.set_option('display.width',170, 'display.max_rows',1000, 'display.max_columns',900)

In [ ]:
df = pd.read_csv("logdata.csv")

In [ ]:
#[match for match in df.columns if "Time" in match]

In [ ]:
pd.to_datetime(df['_indextime'][0],unit='s').tz_localize('utc').tz_convert('Europe/Dublin')

In [ ]:
pd.to_datetime(df['_time'][0],unit='s').tz_localize('utc').tz_convert('Europe/Dublin')

In [ ]:
pd.to_datetime(df['_indextime'][0],unit='s').tz_localize('utc').tz_convert('Europe/Dublin') - pd.to_datetime(df['_time'][0],unit='s').tz_localize('utc').tz_convert('Europe/Dublin')

In [ ]:
#df[['start_time', '_indextime', '_time']]

In [ ]:
df1 = df[['id','start_time', '_indextime', '_time']]

In [ ]:
df1['value'] = df['_indextime'] - df['_time']

In [ ]:
df1['timestamp'] = pd.to_datetime(df1['start_time'])

In [ ]:
df1

### Sort Data 

In [ ]:
df1 = df1.sort_values('timestamp')

In [ ]:
df1.head()

In [ ]:
df1.tail()

In [ ]:
df1['timestamp_diff'] = df1['timestamp'].diff().apply(lambda x: x/np.timedelta64(1, 's')).fillna(0).astype('int64')

Point outliers can be univariate or multivariate, depending on whether they affect one or more time-dependent variables, respectively. 

In [ ]:
plt.figure(figsize = (15,4))
sns.countplot(df1['timestamp_diff']);

### Timestamps are not continous

In [ ]:
data = df1[['id','timestamp', 'value']]

In [ ]:
#data.to_csv("sample_data_randomcut.csv", index = False)

In [ ]:
sns.boxplot(data['value'])

In [ ]:
plt.figure(figsize = (12,4))
sns.lineplot(x = 'timestamp', y = 'value',data = data);

We can see that there is more activity in the morning. Between 9.00 and 9.15 as people start to work. Between 10:30 - 11 am there is not activity after a spike- Tea break. Also Between 12pm - 1 pm nothing much happens

In [ ]:
ts_data = data[['timestamp', 'value']]

In [ ]:
ts_data = ts_data.set_index('timestamp')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.dates as mdates

plt.rc('figure',figsize=(12,8))
plt.rc('font',size=15)

result = seasonal_decompose(ts_data,model='additive', period = int(len(ts_data)/2))
fig = result.plot()

In [ ]:
plt.rc('figure',figsize=(12,6))
plt.rc('font',size=15)

fig, ax = plt.subplots()
x = result.resid.index
y = result.resid.values
ax.plot_date(x, y, color='black',linestyle='--')

ax.annotate('Anomaly', (mdates.date2num(x[35]), y[35]), xytext=(30, 20), 
           textcoords='offset points', color='red',arrowprops=dict(facecolor='red',arrowstyle='fancy'))

fig.autofmt_xdate()
plt.show()

#### Pros

It’s simple, robust, it can handle a lot of different situations, and all anomalies can still be intuitively interpreted.

#### Cons

The biggest downside of this technique is rigid tweaking options. Apart from the threshold and maybe the confidence interval, there isn’t much you can do about it. For example, you’re tracking users on your website that was closed to the public and then was suddenly opened. In this case, you should track anomalies that occur before and after launch periods separately.

### PYOD

* https://neptune.ai/blog/anomaly-detection-in-time-series
* https://pyod.readthedocs.io/en/latest/

Contamination is an important parameter here and I have arrived at its value based on trial and error on validating its results with outliers in 2D plot. It stands for percentage of outlier points in the data.

In [ ]:
data2 = data.value.to_numpy().reshape(-1,1)

In [ ]:
# X_train = np.expand_dims(data['value'][:5000], axis=1)
# X_test = np.expand_dims(data['value'][5000:], axis=1)

In [ ]:
data2_norm = standardizer(data2)

In [ ]:
data2

In [ ]:
from pyod.models.iforest import IForest
# train IForest detector
clf_name = 'IForest'
clf = IForest()
clf.fit(data2_norm)

In [ ]:
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

In [ ]:
data['score'] = y_train_scores


In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

start, end = 0, len(data)

data_subset = data[start:end]

ax1.plot(data_subset["timestamp"], data_subset["value"], color="C0", alpha=0.8)
ax2.plot(data_subset["timestamp"],data_subset["score"], color="C1")

ax1.grid(which="major", axis ="both")

ax1.set_ylabel("Value", color="C0")
ax2.set_ylabel("Anomaly Score", color="C1")

ax1.tick_params("y", color="C0")
ax2.tick_params("y", color="C1")

fig.set_figwidth(10)


In [ ]:
from emmv import emmv_scores
emmv_scores(clf, data2)

### Note we have anomaly score where our eyeball-norm method suggests there is an anomalous data point as well as some places where our eyesballs are not as accurate.


In [ ]:
plt.hist(y_train_scores, bins='auto') 
plt.title("Histogram for Model Anomaly Scores")
plt.show()

If we use a histogram to count the frequency by the anomaly score, we will see the high scores corresponds to a low frequency — evidence of outliers. We choose 0.01 to be the cut point and those >=0.01 to be outliers.

## Plot any data point with scores greater than 3 standard deviations (approximately 99.9th percentile from the mean score).

In [ ]:
score_mean = data['score'].mean()
score_std = data['score'].std()
score_cutoff = score_mean + 3* score_std

anomalies = data_subset[data_subset['score'] > score_cutoff]


### Model Evaluation

In [ ]:
df['is_anomaly'] = np.where(df['id'].isin(list(anomalies['id'])), 1,0)

In [ ]:
eval_df = df[['id', 'is_anomaly']].merge(data[['id', 'y_pred']], on = 'id')

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

accuracy_score(eval_df['is_anomaly'], eval_df['y_pred'])

In [ ]:
confusion_matrix(eval_df['is_anomaly'], eval_df['y_pred'])

In [ ]:
print(classification_report(eval_df['is_anomaly'], eval_df['y_pred']))

In [ ]:
df[df['id'] == 5416]

In [ ]:
data

### Combination Models

In [ ]:
#! pip install combo

"""Example of combining multiple base outlier scores. Four combination
frameworks are demonstrated:
1. Average: take the average of all base detectors
2. maximization : take the maximum score across all detectors as the score
3. Average of Maximum (AOM)
4. Maximum of Average (MOA)
"""

In [ ]:
# standardizing data for processing
X_train = data.value.to_numpy().reshape(-1,1)
X_train_norm = standardizer(X_train)
n_clf = 2  # number of base detectors
# Initialize 20 base detectors for combination
iforest_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140,
           150, 160, 170, 180, 190, 200]

train_scores = np.zeros([X_train.shape[0], n_clf])

for i in range(n_clf):
    iforest = iforest_list[i]

    clf =  IForest(n_estimators=iforest)
    clf.fit(X_train_norm)

    train_scores[:, i] = clf.decision_scores_

In [ ]:
# Decision scores have to be normalized before combination
train_scores_norm = standardizer(train_scores)

In [ ]:
# Combination by average
train_by_average = average(train_scores_norm)
data['score_average'] = train_by_average
score_mean = data['score_average'].mean()
score_std = data['score_average'].std()
score_cutoff = score_mean + 3* score_std

anomalies_avg = data[data['score_average'] > score_cutoff]
(anomalies_avg.shape[0] / df.shape[0]) * 100

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

start, end = 0, len(data)

data_subset = data[start:end]

ax1.plot(data_subset["timestamp"], data_subset["value"], color="C0", alpha=0.8)
ax2.plot(data_subset["timestamp"],data_subset["score_average"], color="C1")

ax1.grid(which="major", axis ="both")

ax1.set_ylabel("Value", color="C0")
ax2.set_ylabel("Anomaly Score", color="C1")

ax1.tick_params("y", color="C0")
ax2.tick_params("y", color="C1")

fig.set_figwidth(10)


In [ ]:
#! pip install emmv